## L1 Language Models, the Chat Format and Tokens
### Course Deeplearning.ai: Building Systems with the ChatGPT API
### URL: https://learn.deeplearning.ai/chatgpt-building-system/

### Flow
#### We need to chain multiple calls to a language model, using different instructions depending on the output of the previous call, and sometimes even looking things up from external sources. 
* Evaluate input to ensure there is no problematic content like hate speech
* Infer query type/intent: complaint, product info request etc.
* Extract relevant info
* Write helpful output
* Check if output is not problematic, innaccurate, or inappropriate

### Two types of LLM
#### **Base LLM**: 
- works by iteratively predicting next word
- e.g. GPT
#### **Instruction tuned LLM**
- trained to answer instructions
- e.g. ChatGPT

#### Training process.
- First the Base LLM is trained on lots of data (100 of billions of words) for months
- The Base LLM is then finetuned on a smaller dataset which consists of instructions and answers. This dataset is often mannually created.
- Once finetuned, the models output are rated by humans on different criteria: correctness, harmful, helpful etc.
- Then the LLM is futher fine tuned to increase the probability of generating the correct output. The common alogrithm for this is RLHF: reinforcement Learning from Human Feedback (RLHF)
- Training an instruction tuned LLM fgrom base LLM requires modest tiem and data compared to the base LLM.

### Setup
#### Load the API key and relevant Python libaries.

In [1]:
import os
import openai
import tiktoken
import tokenizers
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']

#### OpenAI helper function

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [4]:
response = get_completion("What is the capital of France?")
print(response)

The capital of France is Paris.


#### Tokens

In [5]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

The reversed letters of "lollipop" are "pillipol".


#### "lollipop" in reverse should be "popillol"
##### But Chat GPT fails, because it works on tokens
##### Instead if we add - between letters, each letter is a different token and model is able to reverse it

In [28]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")
print(response)

p-o-p-i-l-l-o-l


#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.

In [7]:
### Get encoding scheme
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
encoding

<Encoding 'cl100k_base'>

In [8]:
### print tokens
print(encoding.encode('lollipop'))

[75, 90644]


In [9]:
print(encoding.encode('l-o-l-l-i-p-o-p'))

[75, 16405, 2922, 2922, 18064, 2320, 16405, 2320]


### 1 token is 4 characters or 3/4th of a word.

In [11]:
encoding.decode([75, 16405, 2922, 2922, 18064, 2320, 16405, 2320]

'l-o-l-l-i-p-o-p'

In [14]:
encoding.decode([75, 90644])

'lollipop'

In [18]:
encoding.decode_tokens_bytes([75, 16405, 2922, 2922, 18064, 2320, 16405, 2320])

[b'l', b'-o', b'-l', b'-l', b'-i', b'-p', b'-o', b'-p']

In [19]:
encoding.decode_tokens_bytes([75, 90644])

[b'l', b'ollipop']

## Helper function (chat format)
Here's the helper function we'll use in this course.

In [20]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

In [21]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In a garden so bright, a happy carrot grew,
Beneath the golden sun, its vibrant hue.
With leaves so green, and a smile so wide,
This little carrot brought joy far and wide.

It danced in the breeze, oh what a sight,
Its orange body shining, pure delight.
From garden to table, its journey did go,
Spreading happiness, row after row.

With each crunchy bite, the joy did spread,
From toe to tummy, it filled with widespread.
A happy carrot, oh so grand,
Brings happiness to all, throughout the land.

So remember, my friend, when feeling blue,
Just think of the carrot, so joyful and true.
May its happiness fill your every day,
And keep your worries, far, far away!


In [22]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

Once upon a time, there was a cheerful carrot named Charlie who brightened everyone's day with his vibrant orange hue and infectious laughter.


In [23]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who responds in the style of Dr Seuss. All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

Once upon a time in a garden so grand, there lived a happy carrot with a smile so grand.


In [24]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [25]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [26]:
print(response)

Oh, the happy carrot, so bright and orange,
Grown in the garden, a joyful forage.
With a smile so wide, from top to bottom,
It brings happiness, oh how it blossoms!

In the soil it grew, with love and care,
Nourished by sunshine, fresh air to share.
Its leaves so green, waving in the breeze,
A happy carrot, it aims to please.

With a crunch and a munch, it brings delight,
A healthy snack, oh what a sight!
From garden to plate, it brings such glee,
The happy carrot, a friend to me.

So let's celebrate this veggie so grand,
With a joyful dance, let's all take a stand.
For the happy carrot, so full of cheer,
A simple pleasure, oh how it's dear!


In [27]:
print(token_dict)

{'prompt_tokens': 37, 'completion_tokens': 166, 'total_tokens': 203}


## Notes on using the OpenAI API outside of this classroom